In [1]:
# Class name: AI1811
# Student code: HE181685
# Student name: Nguyễn Thành Trung
# Subject: CPV301-LAB5

In [5]:
import cv2
import numpy as np

def detect_and_match_features(image1, image2):
    # Bước 1: Phát hiện và trích xuất đặc trưng                             
    sift = cv2.SIFT_create()
    
    # Phát hiện điểm đặc trưng và mô tả
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
    
    # Bước 2: Khớp đặc trưng bằng một bộ khớp 
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    
    # Bước 3: Áp dụng kiểm tra tỉ lệ để lọc các cặp khớp (kiểm tra tỉ lệ của Lowe)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    return keypoints1, keypoints2, good_matches

def compute_homography_ransac(keypoints1, keypoints2, matches):
    # Bước 4: Trích xuất các điểm đặc trưng đã khớp
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Bước 5: Tính toán Homography bằng RANSAC
    homography_matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    return homography_matrix, mask

def warp_image(image1, image2, homography_matrix):
    # Bước 6: Biến hình ảnh1 lên ảnh2 sử dụng ma trận homography
    h, w = image2.shape[:2]
    warped_image = cv2.warpPerspective(image1, homography_matrix, (w, h))
    
    return warped_image

# Hàm chính để căn chỉnh hình ảnh
def align_images(image1, image2):
    keypoints1, keypoints2, good_matches = detect_and_match_features(image1, image2)
    
    if len(good_matches) > 4:
        # RANSAC để ước lượng phép biến đổi (homography)
        homography_matrix, mask = compute_homography_ransac(keypoints1, keypoints2, good_matches)
        
        if homography_matrix is not None:
            # Biến hình ảnh1 lên ảnh2
            aligned_image = warp_image(image1, image2, homography_matrix)
            return aligned_image
        else:
            print("Không thể tính toán homography.")
            return None
    else:
        print("Không tìm thấy đủ cặp khớp.")
        return None

# Ví dụ sử dụng
img1 = input("Nhập đường dẫn đến tệp hình ảnh 1: ")
img2 = input("Nhập đường dẫn đến tệp hình ảnh 2: ")
image1 = cv2.imread(img1)
image2 = cv2.imread(img2)
aligned_image = align_images(image1, image2)

if aligned_image is not None:
    window_name = 'Aligned Image'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Đặt cửa sổ để có thể thay đổi kích thước
    cv2.imshow(window_name, aligned_image)
    
    # Đặt cửa sổ kích thước nhỏ hơn
    cv2.resizeWindow(window_name, 800, 600)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu hình ảnh đã căn chỉnh
    cv2.imwrite('aligned_image.jpg', aligned_image)
